In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from tools import data_prep

In [15]:
df1 = data_prep()
df1.info()

D:\Programs\conda\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


<class 'pandas.core.frame.DataFrame'>
Int64Index: 41432 entries, 0 to 41475
Data columns (total 31 columns):
PurchaseID                           41432 non-null int64
PurchaseTimestamp                    41432 non-null int64
PurchaseDate                         41432 non-null object
Auction                              41432 non-null object
VehYear                              41432 non-null float64
Make                                 41432 non-null object
Color                                41432 non-null object
Transmission                         41432 non-null object
WheelTypeID                          41432 non-null object
WheelType                            41380 non-null object
VehOdo                               41432 non-null float64
Nationality                          41432 non-null object
Size                                 41432 non-null object
TopThreeAmericanName                 41432 non-null object
MMRAcquisitionAuctionAveragePrice    41416 non-null object
MMRAcq

In [16]:
# Setting up the dataFrame for Machine learning 
# copy the data and exclude some columns(dropcol) unncessary for training
dropcol = (['PurchaseID','PurchaseTimestamp', 'PurchaseDate' ,'WheelTypeID', 'PRIMEUNIT', \
            'AUCGUART', 'IsOnlineSale', 'ForSale'])
df1 = df1.drop(dropcol,  axis = 1)


In [17]:
#from sklearn.preprocessing import LabelEncoder
#labelEncoder = LabelEncoder()

#categorical_feature_mask = df_final.dtypes==object
#categorical_cols = df_final.columns[categorical_feature_mask].tolist()
#df_final[categorical_cols] = df_final[categorical_cols].apply(lambda col: labelEncoder.fit_transform(col))
#df_final.values[:,7] = labelEncoder.fit_transform(df_final.values[:,7])

#df_final
df1 = data_prep()
# set the random seed - consistent
rs = 10

# train test split
y = df1['IsBadBuy']
X = df1.drop(['IsBadBuy'], axis=1)
X_mat = X.as_matrix()
X_train, X_test, y_train, y_test = train_test_split(X_mat, y, test_size=0.3, stratify=y, random_state=rs)

D:\Programs\conda\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


<class 'pandas.core.frame.DataFrame'>
Int64Index: 41432 entries, 0 to 41475
Data columns (total 31 columns):
PurchaseID                           41432 non-null int64
PurchaseTimestamp                    41432 non-null int64
PurchaseDate                         41432 non-null object
Auction                              41432 non-null object
VehYear                              41432 non-null float64
Make                                 41432 non-null object
Color                                41432 non-null object
Transmission                         41432 non-null object
WheelTypeID                          41432 non-null object
WheelType                            41380 non-null object
VehOdo                               41432 non-null float64
Nationality                          41432 non-null object
Size                                 41432 non-null object
TopThreeAmericanName                 41432 non-null object
MMRAcquisitionAuctionAveragePrice    41416 non-null object
MMRAcq

D:\Programs\conda\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [14]:
# initialise a standard scaler object
scaler = StandardScaler()

# visualise min, max, mean and standard dev of data before scaling
print("Before scaling\n-------------")
for i in range(5):
    col = X_train[:,i]
    print("Variable #{}: min {}, max {}, mean {:.2f} and std dev {:.2f}".
          format(i, min(col), max(col), np.mean(col), np.std(col)))

# learn the mean and std.dev of variables from training data
# then use the learned values to transform training data
X_train = scaler.fit_transform(X_train, y_train)

print("After scaling\n-------------")
for i in range(5):
    col = X_train[:,i]
    print("Variable #{}: min {}, max {}, mean {:.2f} and std dev {:.2f}".
          format(i, min(col), max(col), np.mean(col), np.std(col)))

# use the statistic that you learned from training to transform test data
# NEVER learn from test data, this is supposed to be a set of dataset
# that the model has never seen before
X_test = scaler.transform(X_test)

Before scaling
-------------
Variable #0: min 0, max 41475, mean 20689.33 and std dev 11982.32
Variable #1: min 1231113600, max 1293667200, mean 1262209946.68 and std dev 17954860.16


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [7]:
model = LogisticRegression(random_state=rs)

# fit it to training data
model.fit(X_train, y_train)

/Users/lexi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=10, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [8]:
# training and test accuracy
print("Train accuracy:", model.score(X_train, y_train))
print("Test accuracy:", model.score(X_test, y_test))

# classification report on test data
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


Train accuracy: 0.8708364940348942
Test accuracy: 0.8700724054706356
              precision    recall  f1-score   support

           0       0.87      1.00      0.93     10820
           1       0.31      0.00      0.00      1610

   micro avg       0.87      0.87      0.87     12430
   macro avg       0.59      0.50      0.47     12430
weighted avg       0.80      0.87      0.81     12430



In [9]:
feature_names = X.columns
coef = model.coef_[0]

# limit to 20 features, you can comment the following line to print out everything
coef = coef[:20]

for i in range(len(coef)):
    print(feature_names[i], ':', coef[i])

Transmission : -0.08809261449207823
VehOdo : 0.07697614987079651
MMRAcquisitionAuctionAveragePrice : -0.8952309946189534
MMRAcquisitionAuctionCleanPrice : 0.0900613594528273
MMRAcquisitionRetailAveragePrice : 0.9581828846746545
MMRAcquisitonRetailCleanPrice : -0.07821708586580872
MMRCurrentAuctionAveragePrice : 0.9354512065975777
MMRCurrentAuctionCleanPrice : -0.3210638728159821
MMRCurrentRetailAveragePrice : -0.506398841333568
MMRCurrentRetailCleanPrice : -0.08581751516406032
MMRCurrentRetailRatio : -0.03074918176808289
VehBCost : -0.32712309435490305
WarrantyCost : 0.041484431601290894
Auction_ADESA : 0.07392924609918439
Auction_MANHEIM : -0.0997344382416251
Auction_OTHER : 0.04283257531496547
VehYear_2001.0 : 0.09217908248196918
VehYear_2002.0 : 0.10968796545163366
VehYear_2003.0 : 0.11490531944449085
VehYear_2004.0 : 0.08562868062280296


In [10]:
# grab feature importances from the model and feature name from the original X
coef = model.coef_[0]
feature_names = X.columns

# sort them out in descending order
indices = np.argsort(np.absolute(coef))
indices = np.flip(indices, axis=0)

# limit to 20 features, you can leave this out to print out everything
indices = indices[:20]

for i in indices:
    print(feature_names[i], ':', coef[i])

MMRAcquisitionRetailAveragePrice : 0.9581828846746545
MMRCurrentAuctionAveragePrice : 0.9354512065975777
MMRAcquisitionAuctionAveragePrice : -0.8952309946189534
MMRCurrentRetailAveragePrice : -0.506398841333568
VehBCost : -0.32712309435490305
MMRCurrentAuctionCleanPrice : -0.3210638728159821
VehYear_2008.0 : -0.18381160147654335
WheelType_Alloy : 0.1715457428724775
WheelType_Covers : -0.16969976721996594
Nationality_TOP LINE ASIAN : 0.16804729155914697
TopThreeAmericanName_GM : -0.1475167308037749
Make_NISSAN : -0.12744256469655843
Make_TOYOTA : -0.12096407097233595
VehYear_2003.0 : 0.11490531944449085
VNST_ID : -0.11220067309642154
VehYear_2002.0 : 0.10968796545163366
VehYear_2009.0 : -0.10092824260287092
Auction_MANHEIM : -0.0997344382416251
VehYear_2007.0 : -0.09652881355322182
TopThreeAmericanName_CHRYSLER : 0.09239097537473827


In [11]:
# grid search CV
params = {'C': [pow(10, x) for x in range(-6, 4)]}

# use all cores to tune logistic regression with C parameter
cv = GridSearchCV(param_grid=params, estimator=LogisticRegression(random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train, y_train)

# test the best model
print("Train accuracy:", cv.score(X_train, y_train))
print("Test accuracy:", cv.score(X_test, y_test))

y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred))

# print parameters of the best model
print(cv.best_params_)

/Users/lexi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train accuracy: 0.8708020136542307
Test accuracy: 0.8700724054706356
              precision    recall  f1-score   support

           0       0.87      1.00      0.93     10820
           1       0.22      0.00      0.00      1610

   micro avg       0.87      0.87      0.87     12430
   macro avg       0.55      0.50      0.47     12430
weighted avg       0.79      0.87      0.81     12430

{'C': 0.01}
